**<font face = "Courier New" size = 5 color = #FDFF8F> USO DE MODELO PREENTRENADO  </font>**

<font face = "Courier New" size = 4 > Vamos a trabajar con el modelo pre-entrenado y a obtener predicciones sobre nuestros tweets.
</font>

In [1]:
import numpy as np
import pandas as pd
import os
import re
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns
import pickle
import sqlite3
import numeritos as nitos

pygame 2.1.2 (SDL 2.0.18, Python 3.7.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# CARGAMOS EL MODELO PRE-ENTRENADO

path_my_model  = 'model/sentiment_model'

with open(path_my_model, 'rb') as archivo_entrada:
    my_model = pickle.load(archivo_entrada)

my_model

Pipeline(steps=[('vect',
                 CountVectorizer(max_df=0.5, max_features=1000, min_df=5,
                                 ngram_range=(1, 2))),
                ('cls', LogisticRegression(C=0.5))])

In [3]:
# NOS CONECTAMOS A LA BASE DE DATOS PARA OBTENER LAS TABLAS QUE CONSULTAREMOS

connection = sqlite3.connect('database_twitter.db')
cursor = connection.cursor()

def sql_query(query,cursor):
    cursor.execute(query)
    ans = cursor.fetchall()
    names = [description[0] for description in cursor.description]
    return pd.DataFrame(ans,columns=names)


query_2 = '''SELECT * FROM tweets'''

tweets = sql_query(query_2, cursor)

In [5]:
textos = tweets['tweet']
pd.set_option('display.max_colwidth', 500)
textos

0                                            Recordaros que la semana que viene tenemos la primera quedada #CPS en Madrid, en @TheBridge_Tech, en el que tendremos la gran suerte de contar con @Recuenco para que nos dé una charla. Las plazas se han agotado, pero os podéis apuntar al grupo para próximas quedadas 🚀
1                                                    ¡Vuelven las #becas Digital Talent Pipeline!  1⃣0⃣0⃣ becas para formarte en la próxima edición de nuestros #bootcamp en Madrid, Valencia y Sevilla.  ¡Solicita ya tu beca!🏃‍♀️ https://t.co/V5kikwpJrD  #dtp #becasformacion #digitaltalent  https://t.co/q7RVbD2iLU
2                                                                 El desarrollador web es uno de los perfiles más demandados en el mercado laboral 📈  Descubre nuestro #bootcamp Desarrollo Web Full Stack de la mano de @TheBridge_Tech 💻  Toda la información aquí! 👇  https://t.co/LOoTu97aQq  https://t.co/xpaAzxXSDX
3                                                         

In [6]:
# REALIZAMOS TAREAS DE PROCESADO DE TEXTO Y ELIMINAMOS CARACTERES Y EMOJIS INNECESARIOS. TAMBIÉN ELIMINAREMOS LOS USUARIOS QUE CONSIDERAMOS
# QUE NO APORTAN INFORMACION Y PUEDE CONFUNDIR LOS RESULTADOS.

for i in range(len(textos)):
    if '@' in textos[i]:
        textos[i] ="".join(re.split("@\w+", textos[i]))

In [7]:
# REALIZAMOS TAREAS DE PROCESADO DE TEXTO Y ELIMINAMOS CARACTERES Y EMOJIS INNECESARIOS. TAMBIÉN ELIMINAREMOS LOS USUARIOS QUE CONSIDERAMOS
# QUE NO APORTAN INFORMACION Y PUEDE CONFUNDIR LOS RESULTADOS.

import re

signos = re.compile("(\.)|(\;)|(\:)|(\!)|(\?)|(\¿)|(\@)|(\,)|(\")|(\()|(\))|(\[)|(\])|(\d+)")

def signs_tweets(tweet):
    return signos.sub('', tweet.lower())

textos = textos.apply(signs_tweets)

def remove_links(df):
    return " ".join(['{link}' if ('http') in word else word for word in df.split()])

textos = textos.apply(remove_links)

In [8]:
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

spanish_stopwords = stopwords.words('spanish')

def remove_stopwords(df):
    return " ".join([word for word in df.split() if word not in spanish_stopwords])

    
def spanish_stemmer(x):
    stemmer = SnowballStemmer('spanish')
    return " ".join([stemmer.stem(word) for word in x.split()])

textos = textos.apply(remove_stopwords)
textos = textos.apply(spanish_stemmer)

In [9]:
for i in range(len(textos)):
    textos[i] = nitos.clean_emoji(textos[i])

textos

0                                                              recordar seman vien primer qued #cps madr gran suert cont de charl plaz agot pod apunt grup proxim qued 
1      ¡vuelv #bec digital talent pipelin ⃣⃣⃣ bec formart proxim edicion #bootcamp madr valenci sevill ¡solicit beca‍ {link} #dtp #becasformacion #digitaltalent {link}
2                                               desarroll web perfil demand merc laboral  descubr #bootcamp desarroll web full stack man  tod inform aqu  {link} {link}
3                                                                                                                                                      enhorabuen crack
4            hoy fin ve luz prim cas study public trabaj intens fin bootcamp realiz client real dos seman vertical hic realid mvp invit verl dar feedback {link} {link}
                                                                                     ...                                                                        

In [71]:
# ESTUDIAMOS EL MODELO PRE-ENTRENADO PARA CONOCER LAS VARIABLES USADAS Y LAS MÁS IMPORTANTES

my_model[0].get_feature_names()

['_rubalcaba_',
 'abandon',
 'abc',
 'abc_es',
 'abces',
 'abiert',
 'abog',
 'abraz',
 'abre',
 'abril',
 'absolut',
 'acab',
 'accident',
 'acept',
 'acerc',
 'acert',
 'aclar',
 'acompañ',
 'acord',
 'activ',
 'acto',
 'actual',
 'acuerd',
 'acus',
 'adel',
 'ademas',
 'administr',
 'admir',
 'aeropuert',
 'afect',
 'agu',
 'aguirr',
 'ahi',
 'ahor',
 'ahor si',
 'air',
 'ajust',
 'albert_river',
 'alcald',
 'alegr',
 'alejandrosanz',
 'alemani',
 'alexdelaiglesi',
 'alfons',
 'algui',
 'algun',
 'alli',
 'alons',
 'amaiur',
 'amenaz',
 'amig',
 'amnist',
 'amor',
 'ampli',
 'ana',
 'analisis',
 'andaluc',
 'andaluci',
 'andaluz',
 'anim',
 'anoch',
 'anotici',
 'anterior',
 'anunci',
 'aplaud',
 'aplaus',
 'aplic',
 'apoy',
 'aprob',
 'apuest',
 'aqu',
 'aquell',
 'aqui',
 'aren',
 'argentin',
 'arranc',
 'arte',
 'articul',
 'asegur',
 'asi',
 'asturi',
 'asunt',
 'ataqu',
 'atencion',
 'athletic',
 'audienci',
 'aument',
 'aun',
 'aunqu',
 'auster',
 'autonom',
 'ave',
 'avis',
 

In [72]:
# CREMOS UN DICCIONARO CON TODAS LAS FEATURES Y SU COEFICIENTE DE IMPORTANCIA, LO ORDENAMOS PARA TENER LAS VARIABLES QUE AFECTAN MÁS.

cv = my_model[0]
log_reg = my_model[-1]

feature_to_coef = {
    word: coef for word, coef in zip(
        cv.get_feature_names(), log_reg.coef_[0]
    )
}

for best_negative in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1], 
    reverse=True)[:5]:
    print(best_negative)
    
print('################################')
for best_positive in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1])[:5]:
    print(best_positive)

('cuent', 1.427233666085361)
('deficit', 1.4022058325629765)
('blanc', 1.368604445686843)
('muert', 1.3502570413482133)
('conden', 1.3429067328197288)
################################
('encant', -1.8605111981502893)
('enhorabuen', -1.8449366081495842)
('graci', -1.805329609096349)
('buen', -1.6751185631684895)
('feliz', -1.4730851687952473)


In [ ]:
## PODEMOS VER EN EL OUTPUT ANTERIOR LAS VARIABLES MÁS IMPORTANTES DEL MODELO.

In [10]:
## A CONTINUACIÓN SACAMOS LAS PREDICCIONES SOBRE NUESTROS TWEETS

predictions = my_model.predict(textos)
predictions

array([0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 0], dtype=int64)

In [11]:
# LO MONTAMOS EN UN DATAFRAME PARA VER MEJOR LOS RESULTADOS
 
resultados = pd.DataFrame(tweets['tweet'])
resultados['label'] = predictions



pd.set_option('display.max_colwidth', 500)
pd.set_option('display.max_rows', 500)


resultados


,tweet,label
0,"Recordaros que la semana que viene tenemos la primera quedada #CPS en Madrid, en , en el que tendremos la gran suerte de contar con para que nos dé una charla. Las plazas se han agotado, pero os podéis apuntar al grupo para próximas quedadas 🚀",0
1,"¡Vuelven las #becas Digital Talent Pipeline! 1⃣0⃣0⃣ becas para formarte en la próxima edición de nuestros #bootcamp en Madrid, Valencia y Sevilla. ¡Solicita ya tu beca!🏃‍♀️ https://t.co/V5kikwpJrD #dtp #becasformacion #digitaltalent https://t.co/q7RVbD2iLU",1
2,El desarrollador web es uno de los perfiles más demandados en el mercado laboral 📈 Descubre nuestro #bootcamp Desarrollo Web Full Stack de la mano de 💻 Toda la información aquí! 👇 https://t.co/LOoTu97aQq https://t.co/xpaAzxXSDX,0
3,Enhorabuena crack!,0
4,Hoy por fin ve la luz mi primer Case Study público. Un trabajo intensivo de fin de bootcamp realizado con un cliente real y dos semanas para que las 5 verticales en hiciésemos realidad un MVP. Os invito a verlo y dar feedback. https://t.co/rkCjxulYS9 https://t.co/PqM0dvxj3X,0
5,¡Nuevo acuerdo de colaboración con ! 🤝 👩‍💻Alianza clave en nuestro objetivo de reducir la brecha de competencias digitales y dotar de profesionales técnicos a las empresas. #brechadigital #empleabilidad #digitaltalent https://t.co/tEFiLxMjnA,0
6,"¿Eres o quieres ser talento IT y estás en España? 🇪🇸 Si la respuesta es sí, te contamos que tenemos alianzas con y 💻 Estamos felices abrir nuestras puertas para que estudiantes de nuestros partners se conviertan en futuros Yellowers 🥳",0
7,¡Estamos muy orgullosos de formar equipo junto a ! Participando en la formación de sus futuros empleados con el #bootcamp de #cloud y #devops 🧑‍💻 Ayudando a desarrollar el talento y empleo digital en zonas con alta tasa de paro y pocas oportunidades en el sector #STEM.,1
8,Y seguimos con lo prometido: Primera quedada de la comunidad #CPS en Madrid 🚀: https://t.co/xnWjKu6fPi en un espacio cedido por .,0
9,¡Muchas gracias a por alojarnos! 🔥,0


In [ ]:
import eli5

In [13]:
eli5.show_weights(estimator= my_model,
                  vec= cv,
                  top= 20)

Weight?,Feature
+1.427,cuent
+1.402,deficit
+1.369,blanc
+1.350,muert
+1.343,conden
+1.342,denunci
+1.319,recort
+1.315,indign
+1.310,critic
+1.270,cos


In [ ]:
## EL MODELO TRABAJA CON LA SIGUIENTE ETIQUETA, 0 ES SENTIMIENTO POSITIVO, Y 1 ES NEGATIVO

## VEMOS QUE AL APLICARLO SOBRE NUESTRO TEXTO, ANALIZANDOLO A SIMPLE VISTA, LAS PREDICCIONES NO PARECEN SER MUY CORRECTAS.

## PODRÍAMOS MEJORAR EL MODELO ENTRENANDOLO CON TEXTOS MÁS SIMILARES A CON LOS QUE ESTAMOS TRABAJANDO NOSOTROS. PROBABLEMENTE DE ESTA MANERA 
## PREDICIRÍA MEJOR LOS DATOS QUE LE PASAMOS.

## SERÍA IMPORTANTE TRABAJAR SOBRE LOS DATOS CON LOS QUE SE HA ENTRENADO EL MODELO PREVIAMENTE, PARA ASÍ MEJORAR SU CAPACIDAD.

In [ ]:
## EN EL CASO DE ESTUDIO QUE ESTAMOS LLEVANDO A CABO, SERÍA MUY INTERESANTE PLANTEAR MODELOS DE PREDICCIÓN SOBRE LA REPERCUSIÓN QUE TENDRÍA UN TWEET
## PODRÍAMOS TRABAJAR CON MODELOS DE NLP COMO ESTE, Y TAMBIÉN CREAR OTROS MODELOS BASANDONOS EN DATOS COMO LOS SEGUIDORES DEL USUARIO, LA FECHA DE 
## PUBLICACIÓN DEL TWEET, LAS MÉTRICAS ANTERIORES DE ESA CUENTA, ETC. 

## PODRÍAMOS ESTAR HABLANDO POR EJEMPLO DE UN MODELO DE REGRESIÓN QUE NOS
## PROPORCIONARA LA ESTIMACIÓN DE LIKES QUE TENDRÁ EL TWEET.